In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open('models/many-to-one/adv_trained_models/adv_training.json', 'r') as f:
    data = json.loads(f.read())
    
with open('models/many-to-one/model_info.json', 'r') as f:
    originals = json.loads(f.read())

    
metrics = [
    "R2",
    "RMSE",
    "MAE",
    "MAPE",
]

In [3]:
originals

{'Electricity Transformer Data - 15 min': {'Single Layer LSTM': {'R2': 0.9910858456872373,
   'MAE': 0.25,
   'RMSE': 0.35,
   'MSE': 0.12,
   'MAPE': 4.94,
   'SMAPE': 9.82,
   'MDAPE': 3.08,
   'path': 'models/many-to-one/Electricity Transformer Data - 15 min--Single Layer LSTM.h5'},
  'Double Layer LSTM': {'R2': 0.9924244473768224,
   'MAE': 0.22,
   'RMSE': 0.32,
   'MSE': 0.1,
   'MAPE': 4.34,
   'SMAPE': 9.28,
   'MDAPE': 2.57,
   'path': 'models/many-to-one/Electricity Transformer Data - 15 min--Double Layer LSTM.h5'},
  'Bidirectional LSTM': {'R2': 0.9923119443729341,
   'MAE': 0.23,
   'RMSE': 0.32,
   'MSE': 0.1,
   'MAPE': 4.32,
   'SMAPE': 9.27,
   'MDAPE': 2.63,
   'path': 'models/many-to-one/Electricity Transformer Data - 15 min--Bidirectional LSTM.h5'},
  'CNN1D': {'R2': 0.9791228795450916,
   'MAE': 0.39,
   'RMSE': 0.53,
   'MSE': 0.28,
   'MAPE': 7.8,
   'SMAPE': 13.22,
   'MDAPE': 4.54,
   'path': 'models/many-to-one/Electricity Transformer Data - 15 min--CNN1D.h5'}}

In [4]:
rows = []
for dataset in data.keys():
    for model_name in data[dataset].keys():
        for metric in metrics:
            rows.append((dataset,model_name,metric))
            

columns = [
    ('Original',''), \
    ('FGSM', 'Attack'), ('FGSM', 'Training'), ('FGSM', 'Test'), \
    ('PGD', 'Attack'), ('PGD', 'Training'), ('PGD', 'Test') \
]

row_list = pd.MultiIndex.from_tuples(rows)
col_list = pd.MultiIndex.from_tuples(columns)

A = pd.DataFrame(index = row_list, columns = col_list)

In [5]:
for dataset in originals.keys():
    for model_name in originals[dataset].keys():
        for metric in metrics:
                    A.loc[(dataset, model_name, metric)][('Original', '')] = round(originals[dataset][model_name][metric] ,2)           

In [6]:
for dataset in data.keys():
    for model_name in data[dataset].keys():
        for adv_attack in data[dataset][model_name].keys():
            for setting in data[dataset][model_name][adv_attack].keys():
                if setting == "path": continue
                if "test" in setting:
                    parsed_setting = "Test"
                if "train" in setting:
                    parsed_setting = "Training"
                for metric in metrics:
                    A.loc[(dataset, model_name, metric)][(adv_attack, parsed_setting)] = data[dataset][model_name][adv_attack][setting][metric]

In [7]:
with open("adv_examples/many-to-one/adv_gen_l_inf_2.json") as f:
    attack = json.loads(f.read())
    
for dataset in attack.keys():
    for model_name in attack[dataset].keys():
        for adv_attack in attack[dataset][model_name].keys():
            
            if adv_attack == "FGSM":
                parsed_setting = "epsilon=0.025"
            elif adv_attack == "PGD":
                parsed_setting = "alpha=0.025 | epsilon=0.025"
                
            for metric in metrics:
                A.loc[(dataset, model_name, metric)][(adv_attack, "Attack")] = attack[dataset][model_name][adv_attack][parsed_setting]["metrics"][metric]                


In [8]:
round(A,2).head(50)

Original  \
                                                                         
Electricity Transformer Data - 15 min Single Layer LSTM  R2       0.99   
                                                         RMSE     0.35   
                                                         MAE      0.25   
                                                         MAPE     4.94   
                                      Double Layer LSTM  R2       0.99   
                                                         RMSE     0.32   
                                                         MAE      0.22   
                                                         MAPE     4.34   
                                      Bidirectional LSTM R2       0.99   
                                                         RMSE     0.32   
                                                         MAE      0.23   
                                                         MAPE     4.32   
                                      CNN1D              R2       0.98   
                                                         RMSE     0.53   
                                                         MAE      0.39   
                                                         MAPE      7.8   
Metro Interstate Human Traffic Volume Single Layer LSTM  R2       0.95   
                                                         RMSE   449.74   
                                                         MAE    321.11   
                                                         MAPE    17.21   
                                      Double Layer LSTM  R2       0.95   
                                                         RMSE   433.64   
                                                         MAE    306.83   
                                                         MAPE    19.19   
                                      Bidirectional LSTM R2       0.95   
                                                         RMSE   429.81   
                                                         MAE    305.38   
                                                         MAPE    15.65   
                                      CNN1D              R2       0.95   
                                                         RMSE   448.97   
                                                         MAE    325.54   
                                                         MAPE    19.32   
Beijing-Guanyuan Air-Quality          Single Layer LSTM  R2       0.95   
                                                         RMSE    19.57   
                                                         MAE     10.58   
                                                         MAPE    26.98   
                                      Double Layer LSTM  R2       0.95   
                                                         RMSE    19.67   
                                                         MAE     10.77   
                                                         MAPE    33.26   
                                      Bidirectional LSTM R2       0.95   
                                                         RMSE    19.69   
                                                         MAE     10.79   
                                                         MAPE    34.46   
                                      CNN1D              R2       0.94   
                                                         RMSE    21.56   
                                                         MAE     12.59   
                                                         MAPE    46.36   
Solar Generation - EnerjiSA           Single Layer LSTM  R2       0.99   
                                                         RMSE    12.86   

                                                                 FGSM  \
                                                               Attack   
Electricity Transformer Data - 15 min Single Layer LSTM  R2     0.7

In [9]:
# A.to_csv('adversarial_training_results.csv')

In [10]:
new_A = A.applymap(lambda x: str(x)[0:4] if int(x) < 1 else x)
new_A

Original  \
                                                                            
Electricity Transformer Data - 15 min Single Layer LSTM  R2          0.99   
                                                         RMSE        0.35   
                                                         MAE         0.25   
                                                         MAPE        4.94   
                                      Double Layer LSTM  R2          0.99   
...                                                                   ...   
Solar Generation - EnerjiSA           Bidirectional LSTM MAPE  1550265.83   
                                      CNN1D              R2          0.99   
                                                         RMSE       12.23   
                                                         MAE         7.11   
                                                         MAPE   575486.54   

                                                                      FGSM  \
                                                                    Attack   
Electricity Transformer Data - 15 min Single Layer LSTM  R2           0.77   
                                                         RMSE         1.72   
                                                         MAE           1.7   
                                                         MAPE         32.6   
                                      Double Layer LSTM  R2           0.81   
...                                                                    ...   
Solar Generation - EnerjiSA           Bidirectional LSTM MAPE  13121065.89   
                                      CNN1D              R2           0.92   
                                                         RMSE        37.83   
                                                         MAE          28.7   
                                                         MAPE   3423599.95   

                                                                           \
                                                                 Training   
Electricity Transformer Data - 15 min Single Layer LSTM  R2          0.99   
                                                         RMSE        0.53   
                                                         MAE         0.38   
                                                         MAPE        3.22   
                                      Double Layer LSTM  R2          0.99   
...                                                                   ...   
Solar Generation - EnerjiSA           Bidirectional LSTM MAPE  8379293.62   
                                      CNN1D              R2          0.98   
                                                         RMSE       12.28   
                                                         MAE         7.91   
                                                         MAPE  7936779.84   

                                                                           \
                                                                     Test   
Electricity Transformer Data - 15 min Single Layer LSTM  R2          0.98   
                                                         RMSE        0.49   
                                                         MAE         0.42   
                                                         MAPE        8.47   
                                      Double Layer LSTM  R2          0.99   
...                                                                   ...   
Solar Generation - EnerjiSA           Bidirectional LSTM MAPE   1992684.1   
                                      CNN1D              R2          0.98   
                                                         RMSE       14.97   
                                                         MAE         9.33   
                                                         MAPE  1228233.88   

                                            

In [17]:
new_A.to_excel('adversarial_training_results.xlsx')